In [1]:
import pandas as pd
import numpy as np

# Step 1: Load the Excel file
file_path = "Netflix_data.xlsx"  # Change if needed
df = pd.read_excel(file_path)
print(" Step 1: Loaded Excel file")
print(df.head())

 Step 1: Loaded Excel file
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2 

In [2]:
# Step 2: Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)
print(" Step 2: Cleaned column names")
print(df.columns.tolist())

 Step 2: Cleaned column names
['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [3]:
# Step 3: Handle missing values
df['country'] = df['country'].fillna("Unknown")
df['rating'] = df['rating'].fillna("UNRATED")
df = df.dropna(subset=['title', 'type'])
print("\n Step 3: Handled missing values")
print(df[['country', 'rating']].isnull().sum())

# Step 4: Remove duplicates
df = df.drop_duplicates()
print("\n Step 4: Removed duplicates")
print(f"Remaining rows: {len(df)}")


 Step 3: Handled missing values
country    0
rating     0
dtype: int64

 Step 4: Removed duplicates
Remaining rows: 8807


In [4]:
# Step 5: Standardize text columns
text_cols = ['type', 'title', 'director', 'cast', 'country', 'rating', 'listed_in']
for col in text_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()
print("\n Step 5: Standardized text columns")
print(df[text_cols].head(2))

# Step 6: Convert and clean date_added
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
print("\n Step 6: Converted 'date_added' to datetime")
print(df['date_added'].head())



 Step 5: Standardized text columns
      type                 title         director  \
0    movie  dick johnson is dead  kirsten johnson   
1  tv show         blood & water              nan   

                                                cast        country rating  \
0                                                nan  united states  pg-13   
1  ama qamata, khosi ngema, gail mabalane, thaban...   south africa  tv-ma   

                                         listed_in  
0                                    documentaries  
1  international tv shows, tv dramas, tv mysteries  

 Step 6: Converted 'date_added' to datetime
0   2021-09-25
1   2021-09-24
2   2021-09-24
3   2021-09-24
4   2021-09-24
Name: date_added, dtype: datetime64[ns]


In [5]:
# Step 7: Extract year and month from date_added
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month
print("\n Step 7: Extracted year and month")
print(df[['date_added', 'year_added', 'month_added']].head())

# Step 8: Convert release_year to numeric
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')
print("\n Step 8: Converted 'release_year' to numeric")
print(df['release_year'].describe())


 Step 7: Extracted year and month
  date_added  year_added  month_added
0 2021-09-25      2021.0          9.0
1 2021-09-24      2021.0          9.0
2 2021-09-24      2021.0          9.0
3 2021-09-24      2021.0          9.0
4 2021-09-24      2021.0          9.0

 Step 8: Converted 'release_year' to numeric
count    8807.000000
mean     2014.180198
std         8.819312
min      1925.000000
25%      2013.000000
50%      2017.000000
75%      2019.000000
max      2021.000000
Name: release_year, dtype: float64


In [6]:
# Step 9: Clean duration column
df['duration'] = df['duration'].astype(str).str.strip().str.lower()
df['duration_num'] = df['duration'].str.extract('(\d+)').astype(float)
df['duration_type'] = df['duration'].str.extract('([a-zA-Z]+)')
print("\n Step 9: Cleaned 'duration' column")
print(df[['duration', 'duration_num', 'duration_type']].head())

# Step 10: Remove outliers in duration_num
q1 = df['duration_num'].quantile(0.25)
q3 = df['duration_num'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
df = df[(df['duration_num'] >= lower_bound) & (df['duration_num'] <= upper_bound)]
print("\n Step 10: Removed outliers in 'duration_num'")
print(df['duration_num'].describe())


 Step 9: Cleaned 'duration' column
    duration  duration_num duration_type
0     90 min          90.0           min
1  2 seasons           2.0       seasons
2   1 season           1.0        season
3   1 season           1.0        season
4  2 seasons           2.0       seasons

 Step 10: Removed outliers in 'duration_num'
count    8802.000000
mean       69.796296
std        50.708766
min         1.000000
25%         2.000000
50%        88.000000
75%       106.000000
max       253.000000
Name: duration_num, dtype: float64


In [7]:
# Step 11: Reorder columns
desired_cols = ['show_id', 'type', 'title', 'director', 'country', 'date_added', 'year_added',
                'month_added', 'release_year', 'rating', 'duration', 'duration_num', 'duration_type',
                'listed_in', 'description']
df = df[[col for col in desired_cols if col in df.columns]]
print("\n Step 11: Reordered columns")
print(df.columns.tolist())

# Step 12: Save cleaned data to a new Excel file
df.to_excel("cleaned_netflix_data.xlsx", index=False)
print("\n Step 12: Cleaned file saved as 'cleaned_netflix_data.xlsx'")



 Step 11: Reordered columns
['show_id', 'type', 'title', 'director', 'country', 'date_added', 'year_added', 'month_added', 'release_year', 'rating', 'duration', 'duration_num', 'duration_type', 'listed_in', 'description']

 Step 12: Cleaned file saved as 'cleaned_netflix_data.xlsx'
